In [1]:
import pandas as pd
from smartsheet_dataframe import get_as_df
import os

In [2]:
# Storing Smartsheet information in variables
shitz_token = "wMvGniITjDFd7ClZPE44wtJGvZkM7Hg7mP7if" #Bily's personal token to access G-A smartsheet data. Anyone can obtain their own token
fhaid = 4523924143794052 # The ID number that references the FHA sheet
mssid = 3005704744265604 # The ID number that references the MSS sheet

In [3]:
# Creating the FHA and MSS dataframes
fha_df = get_as_df(type_='sheet',
                  token=shitz_token,
                  id_=fhaid)

mss_df = get_as_df(type_='sheet',
                  token=shitz_token,
                  id_=mssid)

In [4]:
war_df = mss_df.loc[mss_df['UniqueID'].str.contains("WAR")].copy()

In [5]:
client_df = war_df[['UniqueID',
                    'Tract',
                    'CMP',
                    'Stand',
                    'Acres',
                    'Age',
                    'County',
                    'State',
                    'Stand Type',
                    'Origin',
                    'Scheduled',
                    'PreBA',
                    'PreTPA',
                    'PreGTA',
                    'PlnBA',
                    'PlnTPA',
                    'PlnGTA',
                    'Client GT',
                    'Future GT',
                    'Reporting Period']].copy()

client_df['Reporting Period'] = client_df['Reporting Period'].fillna(0)
client_df.loc[client_df['Reporting Period'] == "", 'Reporting Period'] = 0
s = client_df.loc[pd.to_numeric(client_df['Reporting Period'], errors='coerce').isnull() & client_df['Reporting Period'].str.contains("RP"), 'Reporting Period'].apply(lambda x: x.strip()[-1])
client_df['trial'] = client_df.index.map(s)
client_df.loc[pd.to_numeric(client_df['Reporting Period'], errors='coerce').isnull() & client_df['Reporting Period'].str.contains("RP"), 'Reporting Period'] = client_df.loc[pd.to_numeric(client_df['Reporting Period'], errors='coerce').isnull() & client_df['Reporting Period'].str.contains("RP"), 'trial']
client_df['Reporting Period'] = pd.to_numeric(client_df['Reporting Period'], downcast='integer')
client_df['Age'] = pd.to_numeric(client_df['Age'], errors='coerce', downcast='integer').fillna(0).astype(int)
client_df.loc[pd.to_numeric(client_df['Future GT'], errors='coerce').isnull(), 'Future GT'] = 0
client_df['Tract'] = client_df['Tract'].astype(str)
client_df['Stand'] = client_df['Stand'].astype(str)
client_df['Tract'] = client_df['Tract'].apply(lambda x: x.split('.')[0])
client_df['Client GT'] = pd.to_numeric(client_df['Client GT'], errors="coerce")
client_df['Client GT'] = client_df['Client GT'].fillna(0)

cols = ['PreBA', 'PreTPA', 'PreGTA', 'PlnBA', 'PlnTPA', 'PlnGTA']

for x in cols:
    client_df[x] = pd.to_numeric(client_df[x], errors='coerce', downcast='integer').fillna(-99).astype(int)

client_df.drop(columns=['trial'],inplace=True)


I am thinking of creating more tables : county, state, origin, species. That way if we have to update the way somethign is spelled we just do it in one place.

Maybe even tract as well?

In [6]:
species = ["Slash Pine", "Loblolly Pine", 'Mixed Natural Pine', 'Hardwood Mix', 'Pine Straw', "Other"]
species_dict = {'Species': species}
species_df = pd.DataFrame(species_dict)
species_df['SpeciesUID'] = species_df.index

def species_map(x):
    x = x.lower()
    if "slash" in x:
        return 0
    elif "loblol" in x:
        return 1
    elif "mixed natural" in x:
        return 2
    elif "h" in x:
        return 3
    elif "straw" in x:
        return 4
    elif "pine" in x:
        return 1
    else:
        return 5
    
client_df['Stand Type'] = client_df['Stand Type'].apply(lambda x: species_map(x))


In [7]:
origins = ['Planted', 'Natural', 'Natural Mix', 'Other']
origin_dict = {'Origin': origins}
origin_df = pd.DataFrame(origin_dict)
origin_df['OriginUID'] = origin_df.index

def origin_map(x):
    x = x.lower()
    if "plant" in x:
        return 0
    elif "natural mix" in x:
        return 2
    elif "nat" in x:
        return 1
    else:
        return 3
client_df['Origin'] = client_df['Origin'].apply(lambda x: origin_map(x))


In [8]:
ga_intake_df = war_df[['IntakeID',
                       'FolderID',
                       'UniqueID',
                       'GA_ACRES',
                       'GA GT', # this is going to be GA total isn't it? Or does it stay as GA estimate of current GT (total minus what client says is future)
                       'Total GT', # Does this need included? At the moment we do funny stuff with total gt and subtracting future gt to get GA current rp (above)
                       'Notes'
                       ]].copy()

# Do I want to include a Ticketid column here or use a crossreference table?
# Should reporting period be in here?

In [9]:
status_df = war_df[['UniqueID',
                    'Guidance',
                    'Expiration',
                    'GuidanceProvider',
                    'Status',
                    'Completed Date'
                    ]].copy()

In [10]:
def intake_breaker(lst):
    intake_lst = []
    activity_lst = []
    for ele in lst:
        intake_lst.append(ele[:9])
        activity_lst.append(ele[9:11])
        intake_st = set(intake_lst)
        activity_st = set(activity_lst)
        
    return intake_lst, activity_lst, intake_st, activity_st


def ticketid(lst):
    """This function creates a ticket id based off a list of unique ids

    Args:
        lst (list): a list of unique ids that have the same Tract_Cmp_Stand from MSS
    """
    if len(lst) == 0:
        return "ERROR, Where is your uniqueid?"
    elif len(lst) < 2:
        return lst[0]
    elif len(lst) > 1:
        # Check to see if there are multiple intakes
        temp_lst, activity_lst, temp_st, activity_st = intake_breaker(lst)
        temp_lst.sort()
        activity_lst.sort()
        if len(temp_st) > 1:
            return f"{temp_lst[0]}XXXX"
        # If there are not multiple intakes. Check to see if there are multiple activities in the lst (Intake portion should be same for all)
        elif len(activity_st) > 1:
           return f"{temp_lst[0]}XXXX"
       # If there is only a single activity type
        elif len(activity_st) == 1:
            return f"{temp_lst[0]}{activity_lst[0]}XX"
        else:
            return "Failed within len(lst) > 1"
        
    else:
        return "Failed at length of lst" 

In [11]:
war_inprog = war_df[war_df['Status'].isin(['OPEN', 'IN PROCESS', 'POST', 'PENDING'])].copy()
war_inprog['Tract'] = war_inprog['Tract'].astype(str).apply(lambda x: x.split('.')[0])
war_inprog['CMP'] = war_inprog['CMP'].astype(str).apply(lambda x: x.split('.')[0])
war_inprog['Stand'] = war_inprog['Stand'].astype(str).apply(lambda x: x.split('.')[0])
war_inprog['TCS'] = war_inprog['Tract'] + "_" + war_inprog['CMP'] + "_" + war_inprog['Stand']

# After talking with Rob, we need to make sure that we separate out the GA and the ARB Guidance
grouped_activities = war_inprog.groupby(['TCS', 'GuidanceProvider'])['UniqueID'].apply(list).reset_index()

trial_df = grouped_activities.copy()
trial_df['BBID'] = trial_df['UniqueID'].apply(lambda x: ticketid(x))

prev_combined = war_inprog.groupby('TicketID')['TCS'].apply(set).reset_index()

tcs = []
for val in prev_combined['TCS'].loc[prev_combined['TCS'].str.len()> 1].values:
    for item in val:
        tcs.append(item)
        
ticket_id_dict = {}
for i, row in prev_combined.loc[prev_combined['TCS'].str.len()> 1].iterrows():
    # print(row['TicketID']) # ticket id
    value_list = []
    for value in row['TCS']:
        value_list.append(value)
    ticket_id_dict[row['TicketID']] = value_list
    
for k, v in ticket_id_dict.items():
    trial_df.loc[trial_df['TCS'].isin(v), 'otherid'] = k

def final_id(row):
    if row['otherid'] != "nan":
        return row['otherid']
    else:
        return row['BBID']
    
trial_df['finalid'] = trial_df[['BBID', 'otherid']].apply(final_id, axis=1)

def break_list(x):
    trial_list = []
    for item in x:
        for iter in item:
            trial_list.append(iter)
            
    return trial_list

new_ticket_df = trial_df.groupby('finalid').agg({'TCS': lambda x: list(x),
                                 'UniqueID': lambda x: break_list(x)}).reset_index()

client_for_ticket = client_df.copy()
for id in new_ticket_df['finalid'].unique().tolist():
    client_for_ticket.loc[client_for_ticket['UniqueID'].isin(new_ticket_df.loc[new_ticket_df['finalid'] == id, 'UniqueID'].values[0]), 'TicketID'] = id

new_tcs_intake = client_for_ticket.groupby(['TicketID',
                                            'UniqueID'])['Client GT'].sum().reset_index()

old_ticket_ids = dict(zip(war_df['UniqueID'], war_df['TicketID']))
new_tcs_intake['old_ticketids'] = new_tcs_intake['UniqueID'].map(old_ticket_ids)
new_tcs_intake.loc[~new_tcs_intake['TicketID'].str.contains("WAR"), 'TicketID'] = new_tcs_intake['old_ticketids']

# Manually fix this issue where we had to deal with overlaps and SPB management
new_tcs_intake.loc[new_tcs_intake['UniqueID'] == 'WAR231006BI02', 'TicketID'] = "WAR230817CCXX"
new_tcs_intake.loc[new_tcs_intake['UniqueID'] == 'WAR240627BI01', 'TicketID'] = 'WAR230908XXXX'
new_tcs_intake.loc[new_tcs_intake['UniqueID'] == 'WAR180518TT01', 'TicketID'] = 'WAR180518XXXX'
new_tcs_intake.loc[new_tcs_intake['UniqueID'] == 'WAR180518TH03', 'TicketID'] = 'WAR180518XXXX'

ticketid_table = new_tcs_intake[['TicketID', 'UniqueID']]

## Getting the ticket table

In [12]:
ticketid_table[ticketid_table['UniqueID'] == "WAR240627BI01"]

,TicketID,UniqueID
97,WAR230908XXXX,WAR240627BI01


In [13]:
mill_loc = r"P:\Mill Tickets"
file_pattern = "WAR_RP*_Compiled.xlsm"
big_df = pd.DataFrame()

mill_name = []
for root, dirs, files in os.walk(mill_loc):
    for file in files:
        if "WAR_RP" in file and file.endswith("Compiled.xlsm"):
            if "$" in file:
                pass
            else:
                print(root)
                print(file)
                df = pd.read_excel(os.path.join(root, file), sheet_name='RP_Project')
                big_df = pd.concat([big_df, df])
                

big_df.loc[big_df['Tons (U.S)'].isnull(), 'Tons (U.S)'] = big_df.loc[big_df['Tons (U.S)'].isnull(), 'TonsUS']
big_df['Cull Tons'].fillna(big_df['CullTons'], inplace=True)
big_df['Accepted Tons'].fillna(big_df['AcceptedTons'], inplace=True)
big_df['Reporting Period'].fillna(big_df['ReportingPeriod'],inplace=True)

def rep_fix(x):
    if "RP" in str(x):
        return x[-1]
    else:
        return x
    
big_df['Reporting Period'] = big_df['Reporting Period'].apply(lambda x: rep_fix(x))
big_df['Reporting Period'] = pd.to_numeric(big_df['Reporting Period'])

big_df['Date'] = pd.to_datetime(big_df['Date'])

mill_df = big_df[['Date',
                  'Ticket',
                  'Mill',
                  'Product',
                  'Tons (U.S)',
                  'Cull Tons',
                  'Accepted Tons',
                  'Source',
                  'Notes',
                  'Reporting Period',
                  'TicketID',
                  'Week']].copy()

mill_df = mill_df.drop_duplicates()

P:\Mill Tickets\ARCHIVE\WAR_RP6
WAR_RP6_Compiled.xlsm
P:\Mill Tickets\ARCHIVE\WAR_RP7
WAR_RP7_Compiled.xlsm
P:\Mill Tickets\ARCHIVE\WAR_RP8
WAR_RP8_Compiled.xlsm
P:\Mill Tickets\Retroactive_Ticket_imageing\WAR\WAR_RP2
WAR_RP2_Compiled.xlsm


c:\Users\bbrown\Anaconda3\envs\postgres_python\Lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


P:\Mill Tickets\Retroactive_Ticket_imageing\WAR\WAR_RP3
WAR_RP3_Compiled.xlsm


c:\Users\bbrown\Anaconda3\envs\postgres_python\Lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


P:\Mill Tickets\Retroactive_Ticket_imageing\WAR\WAR_RP4
WAR_RP4_Compiled.xlsm


c:\Users\bbrown\Anaconda3\envs\postgres_python\Lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


P:\Mill Tickets\Retroactive_Ticket_imageing\WAR\WAR_RP5
WAR_RP5_Compiled.xlsm


c:\Users\bbrown\Anaconda3\envs\postgres_python\Lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


P:\Mill Tickets\WAR_RP9
WAR_RP9_Compiled.xlsm


# Getting the expected harvest

Find all activities that are Open, Pending, In Process, Post

Get their ticket id

Get the sum of the expected rp8 for that ticketid
get the sum of the expected rp9 for that ticketid

get the harvested volumes for those ticketids (already did this by getting the sums and translating from old ticketid to new ticketid)

### First Step

Make the base expected table

I think we need to start this from the MSS_df

To get us values as stated on MSS:
- Get the Expected GT for the RP requested
- Get the Future GT
  - Add 1 to the RP requested
- Groupby the new Ticket ID

To Updated for RP9:
- get all ticket ids that are open in process, pending, post
- Get the harvested volume for those ticket ids (might have to walk from old ticketid to uniqueid to new ticketid)
- find any remaining expected for those ticket ids
  - Add remaining to the RP9 volume




Get harvested tons by old tid

In [14]:
harvested_tons_oldTID = mill_df[mill_df['Reporting Period'] == 8].groupby("TicketID")['Tons (U.S)'].sum().reset_index()

In [15]:
harvested_tons_oldTID['Tons (U.S)'].sum()

345687.7569999999

Let's get new and old TIDs that are currently in process

In [16]:
# Getting harvest ids that were in RP 8 and are still "Active" (Potentially need updating due to harvest)
harvest_activities = war_df[(war_df['Reporting Period'] == 8) & (war_df['Status'].isin(['IN PROCESS', "POST"]))]['UniqueID'].unique().tolist()
harvest_old_ticketid = war_df[(war_df['Reporting Period'] == 8) & (war_df['Status'].isin(['IN PROCESS', "POST"]))]['TicketID'].unique().tolist()

In [17]:
# Let's see how many tons are under old TIDs of activities in question
harvested_tons_oldTID[harvested_tons_oldTID['TicketID'].isin(harvest_old_ticketid)]['Tons (U.S)'].sum()

146413.92999999996

In [18]:
# Let's get the new TIDs based on these harvest activities
new_ticketIDs = ticketid_table[ticketid_table['UniqueID'].isin(harvest_activities)]['TicketID'].unique().tolist()


In [19]:
# Let's see what is in old_ticketid that is not in new_ticketIDs
[item for item in harvest_old_ticketid if item not in new_ticketIDs]

['WAR230818TH01']

Ok, so it looks like we don't have any old ticketids in the harvested file that are not in the new ticketids

That doesn't mean there aren't changes of uniqueids between the two sets of ticketids

In [20]:
war_df[war_df['TicketID'] == "WAR220728XXXX"]['UniqueID'].unique().tolist()

['WAR220728TT01', 'WAR220728TH02']

In [21]:
ticketid_table[ticketid_table['TicketID'] == "WAR220728XXXX"]

,TicketID,UniqueID
205,WAR220728XXXX,WAR220728TH02
206,WAR220728XXXX,WAR220728TT01


In [22]:
for id in harvest_old_ticketid:
    old_unique = war_df[war_df['TicketID'] == id]['UniqueID'].unique().tolist()
    new_unique = ticketid_table[ticketid_table['TicketID'] == id]['UniqueID'].unique().tolist()
    inold_notinew = [item for item in old_unique if item not in new_unique]
    if len(inold_notinew) > 0:
        print(id, old_unique)

WAR230818TH01 ['WAR230818TH01']


Ok, so it looks like we are lucky and the new ticketids aren't messing up the accounting

## Next steps

- Get the expected GT for RP8 by new ticketid
- Get the expected GT for RP9 by new ticketid
- Update the expected RP9 GT with Rob's harvesting ruleset

RP8 Expected - will be the conservative between Client and GT (in war_df it will be the "Current RP Expected GT")


In [23]:
trial_expected = pd.merge(ticketid_table, war_df[['UniqueID', 'Current RP Expected GT', 'Status', 'Future GT']], on='UniqueID', how='left')
trial_expected = pd.merge(trial_expected, client_df[['UniqueID', 'Reporting Period']], on='UniqueID', how='left')
war_rp8_expected = trial_expected[(trial_expected['Reporting Period'] == 8) & trial_expected['Status'].isin(['IN PROCESS', 'OPEN', 'POST'])].groupby('TicketID')['Current RP Expected GT'].sum().reset_index()
war_rp8_expected = pd.merge(war_rp8_expected, harvested_tons_oldTID, on='TicketID', how='left')
war_rp8_expected['Tons (U.S)'] = pd.to_numeric(war_rp8_expected['Tons (U.S)'], errors='coerce', downcast='float')
war_rp8_expected['Tons (U.S)'].fillna(0, inplace=True)
war_rp8_expected['Remaining'] = war_rp8_expected['Current RP Expected GT'] - war_rp8_expected['Tons (U.S)']

In [24]:
war_rp8_expected[war_rp8_expected['Remaining'] > 0]

,TicketID,Current RP Expected GT,Tons (U.S),Remaining
0,WAR230113TD03,7322.0,0.000000,7322.0
1,WAR230113TD04,12680.0,11999.750000,680.25
3,WAR230303TH01,4253.0,0.000000,4253.0
6,WAR230318TT04,1235.96,0.000000,1235.96
8,WAR230318TT07,828.0,0.000000,828.0
9,WAR230711TH01,15300.0,15124.259766,175.740234
10,WAR230712TH01,4450.0,1811.500000,2638.5
11,WAR230712TH02,3312.39,2347.729980,964.66002
13,WAR230818XXXX,3250.45,0.000000,3250.45
14,WAR230908TH04,18043.550781,13091.679688,4951.871094


In [25]:
# war_rp8_expected.to_excel('rp8_accounting.xlsx')

In [26]:
war_rp9_expected = trial_expected[(trial_expected['Reporting Period'] == 9) & (trial_expected['Status'].isin(['OPEN', 'IN PROCESS', 'PENDING', 'POST']))].groupby('TicketID')['Current RP Expected GT'].sum().reset_index()
war_rp9_expected_1 = trial_expected[(trial_expected['Reporting Period'] == 8) & (trial_expected['Status'].isin(['OPEN', 'IN PROCESS', 'PENDING', 'POST']))].groupby('TicketID')['Future GT'].sum().reset_index()

In [27]:
war_rp9_expected = pd.merge(war_rp9_expected, war_rp9_expected_1, on='TicketID', how='outer')

In [28]:
war_rp9_expected['Current RP Expected GT'] = pd.to_numeric(war_rp9_expected['Current RP Expected GT'], errors='coerce', downcast='float')
war_rp9_expected['Future GT'] = pd.to_numeric(war_rp9_expected['Future GT'], errors='coerce', downcast='float')
war_rp9_expected.fillna(0, inplace=True)
war_rp9_expected['RP9_Expected'] = war_rp9_expected['Current RP Expected GT'] + war_rp9_expected['Future GT']
war_rp9_expected

,TicketID,Current RP Expected GT,Future GT,RP9_Expected
0,WAR230818XXXX,2699.61,0.000000,2699.610000
1,WAR230908XXXX,28182.89,0.000000,28182.890000
2,WAR231107XXXX,9891.59,0.000000,9891.590000
3,WAR240521CCXX,9814.68,0.000000,9814.680000
4,WAR240521TN01,2918.65,0.000000,2918.650000
...,...,...,...,...
67,WAR240606TH03,0.00,2810.100098,2810.100098
68,WAR240606TH04,0.00,878.400024,878.400024
69,WAR240618BI01,0.00,0.000000,0.000000
70,WAR240618BI02,0.00,0.000000,0.000000


In [29]:
# war_rp9_expected.to_excel("rp9_investigate.xlsx")

In [30]:
ticketid_table[ticketid_table['UniqueID'] == 'WAR231006BI02']

,TicketID,UniqueID
20,WAR230817CCXX,WAR231006BI02


In [31]:
# ticketid_table.to_excel("TicketID_Table.xlsx")

In [32]:
# new_tcs_intake.to_excel("new-to-oldtids.xlsx")

In [33]:
# new_ticket_df.to_excel("ticketlookip_v2.xlsx")

In [34]:
war_trial = war_df.copy()

In [35]:
war_trial = war_trial[war_trial['Status'].isin(['IN PROCESS', 'OPEN', 'PENDING', 'POST'])].copy()

In [36]:
war_trial['Current RP Expected GT'] = pd.to_numeric(war_trial['Current RP Expected GT'], errors='coerce').fillna(0)
war_trial['Future GT'] = pd.to_numeric(war_trial['Future GT'], errors='coerce').fillna(0)
war_trial['Total GT'] = pd.to_numeric(war_trial['Total GT'], errors='coerce').fillna(0)

In [37]:
war_trial = war_trial[['TicketID', 'Current RP Expected GT', 'Future GT', 'Total GT']]

In [38]:
war_trial['Total GT'].sum()

543049.456834

In [39]:
war_totals = war_trial.groupby('TicketID').agg({
    "Current RP Expected GT": "sum",
    'Future GT': "sum",
    "Total GT": "sum"
}).reset_index()

In [40]:
war_totals

,TicketID,Current RP Expected GT,Future GT,Total GT
0,WAR230113TD03,7322.00,0.0,7322.00
1,WAR230113TD04,12680.00,0.0,12680.00
2,WAR230203TTXX,4186.00,0.0,0.00
3,WAR230303TH01,4253.00,0.0,4253.00
4,WAR230318CC05,4256.00,0.0,4256.00
...,...,...,...,...
68,WAR241025HD09,4781.69,0.0,4781.69
69,WAR241025HD16,5174.26,0.0,5180.00
70,WAR241025HD17,4689.54,0.0,4690.00
71,WAR241025HDXX,21772.72,0.0,21772.72


In [41]:
ticket_harvested = mill_df.groupby(['TicketID']).agg({'Tons (U.S)': "sum",
                                                      'Date': "max"})

In [42]:
ticket_harvested

,Tons (U.S),Date
TicketID,,
WAR180206TH01,5145.14,2018-10-16
WAR180206TH02,6705.43,2018-07-26
WAR180206TH03,5644.90,2019-05-15
WAR180518TH01,2514.31,2018-10-03
WAR180518TH02,2258.22,2018-08-16
...,...,...
WAR240302TT03,2510.22,2024-06-14
WAR240417TH01,3893.61,2024-08-10
WAR240606TH02,5270.54,2024-06-26


In [43]:
war_figure = pd.merge(war_totals, ticket_harvested, on='TicketID', how='left')

In [44]:
# war_figure.to_excel("total_expected_3.xlsx")

In [45]:
new_ticket_df['tcsLookup'] = new_ticket_df['TCS'].apply(lambda x: x[0])

In [46]:
new_ticket_df

,finalid,TCS,UniqueID,tcsLookup
0,WAR230113TD03,"[13180_NP_13, 18, 23, &27]",[WAR230113TD03],"13180_NP_13, 18, 23, &27"
1,WAR230113TD04,[33094A & 13180_NP_1 & 2],[WAR230113TD04],33094A & 13180_NP_1 & 2
2,WAR230203TTXX,"[13206_024 018_13, 13206_024 018_14]","[WAR230203TT01, WAR230203TT02]",13206_024 018_13
3,WAR230303TH01,[Clinch County_1_2],[WAR230303TH01],Clinch County_1_2
4,WAR230318CC05,[BW55_0040 0009_CC1],[WAR230318CC05],BW55_0040 0009_CC1
...,...,...,...,...
67,WAR241025HD09,[Clinch_3_10],[WAR241025HD09],Clinch_3_10
68,WAR241025HD16,[Clinch_7_5],[WAR241025HD16],Clinch_7_5
69,WAR241025HD17,[Clinch_7_6],[WAR241025HD17],Clinch_7_6
70,WAR241025HDXX,"[Clinch_6_1, Clinch_6_2, Clinch_6_3, Clinch_6_...","[WAR241025HD13, WAR241025HD14, WAR241025HD10, ...",Clinch_6_1


In [47]:
mill_df

,Date,Ticket,Mill,Product,Tons (U.S),Cull Tons,Accepted Tons,Source,Notes,Reporting Period,TicketID,Week
0,2021-08-02,160048,Norbord Inc - Huguley,PPW,26.31,0.000,26.310,M-82_Week 1.pdf,NaN,6,WAR201030TH04,Week01
1,2021-08-03,160157,Norbord Inc - Huguley,PPW,25.79,0.000,25.790,M-82_Week 1.pdf,NaN,6,WAR201030TH04,Week01
2,2021-08-03,160120,Norbord Inc - Huguley,PPW,25.32,0.000,25.320,M-82_Week 1.pdf,,6,WAR201030TH04,Week01
3,2021-08-03,160155,Norbord Inc - Huguley,PPW,25.31,0.000,25.310,M-82_Week 1.pdf,,6,WAR201030TH04,Week01
4,2021-08-03,160082,Norbord Inc - Huguley,PPW,27.17,0.000,27.170,M-82_Week 1.pdf,,6,WAR201030TH04,Week01
...,...,...,...,...,...,...,...,...,...,...,...,...
1569,2024-08-16,292262,West Fraser - Huguley,PPW,27.93,0.000,27.930,13093 week 20 pine beetle salvage (Flint Crew...,NaN,9,WAR240618BI03,Week10
1570,2024-08-17,292264,West Fraser - Huguley,PPW,29.21,0.000,29.210,13093 week 20 pine beetle salvage (Flint Crew...,NaN,9,WAR240618BI03,Week10
1571,2024-08-16,69692,Interfor - Thomaston,PST,31.24,0.125,31.115,13093 week 20 pine beetle salvage (Flint Crew...,NaN,9,WAR240618BI03,Week10
1572,2024-08-16,69753,Interfor - Thomaston,PST,27.99,0.000,27.990,13093 week 20 pine beetle salvage (Flint Crew...,NaN,9,WAR240618BI03,Week10


## Build DB

All tables need a primary key column created/inserted

- [x] Client Provided info
- [x] GA Gas intake info
- [x] ticketid table
- [x] ticketid/uniqueid lookup table - Need to change the ticketid to the ticketid table primary key (map)
- [x] tree species table
- [x] tree origin table
- [x] RP expected table
- [/] Mill_ticket_table - needs edited/3NF
- [/] Status table - need to map guidance provider primary key
- [ ] Guidance provider table
- [ ] Activity_Plots Table

In [48]:
client_df.head()

,UniqueID,Tract,CMP,Stand,Acres,Age,County,State,Stand Type,Origin,Scheduled,PreBA,PreTPA,PreGTA,PlnBA,PlnTPA,PlnGTA,Client GT,Future GT,Reporting Period
0,WAR241025TT01,WACO,4255,1,150.92,17,Crawford,GA,1,0,,110,500,110,70,300,80,4528.00,0.0,9
1,WAR241025HD17,Clinch,7,6,67.0,40,Clinch,GA,0,0,,130,200,70,0,0,0,4675.00,0.0,9
2,WAR241025HD16,Clinch,7,5,74.0,40,Clinch,GA,0,0,,130,200,70,0,0,0,5158.00,0.0,9
3,WAR241025HD15,Clinch,6,5,5.0,42,Clinch,GA,0,0,,130,200,70,0,0,0,350.04,0.0,9
4,WAR241025HD14,Clinch,6,2,62.9,42,Clinch,GA,0,0,,130,200,70,0,0,0,4403.53,0.0,9


In [49]:
species_df

,Species,SpeciesUID
0,Slash Pine,0
1,Loblolly Pine,1
2,Mixed Natural Pine,2
3,Hardwood Mix,3
4,Pine Straw,4
5,Other,5


In [50]:
origin_df

,Origin,OriginUID
0,Planted,0
1,Natural,1
2,Natural Mix,2
3,Other,3


In [51]:
# This needs to be just ticketid as a foreign key and then the index become a column that is the primary key and that will be referenced elsewhere.
ticketid_lookup_table = ticketid_table.copy()

In [52]:
ticketids = ticketid_table['TicketID'].unique().tolist()
ticketids_dict = {}
ticketids_dict['TicketID'] = ticketids
ticket_id_df = pd.DataFrame(ticketids_dict)
ticket_id_df['TicketUID'] = ticket_id_df.index
ticket_mapper = dict(zip(ticket_id_df['TicketID'], ticket_id_df['TicketUID']))
ticketid_lookup_table['TicketUID'] = ticketid_lookup_table['TicketID'].map(ticket_mapper)
ticketid_lookup_table.drop(columns='TicketID', inplace=True)

In [53]:
ticketid_lookup_table

,UniqueID,TicketUID
0,WAR230113TD03,0
1,WAR230113TD04,1
2,WAR230203TT01,2
3,WAR230203TT02,2
4,WAR230303TH01,3
...,...,...
291,WAR240529BI07,201
292,WAR240529TT01,202
293,WAR240719CC01,203
294,WAR240719CC02,204


In [54]:
ticket_id_df

,TicketID,TicketUID
0,WAR230113TD03,0
1,WAR230113TD04,1
2,WAR230203TTXX,2
3,WAR230303TH01,3
4,WAR230318CC05,4
...,...,...
201,WAR240529BI07,201
202,WAR240529TT01,202
203,WAR240719CC01,203
204,WAR240719CC02,204


In [55]:
ga_intake_df

,IntakeID,FolderID,UniqueID,GA_ACRES,GA GT,Total GT,Notes
0,WAR241025,WAR241025a,WAR241025TT01,134.66,4039.86,4528.0,
1,WAR241025,WAR241025b,WAR241025HD17,66.99,4689.54,4690.0,
2,WAR241025,WAR241025b,WAR241025HD16,73.92,5174.26,5180.0,
3,WAR241025,WAR241025b,WAR241025HD15,5.01,350.58,350.58,
4,WAR241025,WAR241025b,WAR241025HD14,63.11,4417.97,4417.97,
...,...,...,...,...,...,...,...
589,WAR180518,WAR180518,WAR180518TH01,140.7,,,
590,WAR180206,WAR180206,WAR180206TH04,,,,is a duplicate of WAR190311
591,WAR180206,WAR180206,WAR180206TH03,,,,is a duplicate of WAR190311
592,WAR180206,WAR180206,WAR180206TH02,116.0,,,Pulled off due to tropical depression. Too wet.


In [56]:
mill_df.loc[mill_df['TicketID'] == "WAR18051XXXX", 'TicketID'] = "WAR180518XXXX"

In [57]:
mill_df

,Date,Ticket,Mill,Product,Tons (U.S),Cull Tons,Accepted Tons,Source,Notes,Reporting Period,TicketID,Week
0,2021-08-02,160048,Norbord Inc - Huguley,PPW,26.31,0.000,26.310,M-82_Week 1.pdf,NaN,6,WAR201030TH04,Week01
1,2021-08-03,160157,Norbord Inc - Huguley,PPW,25.79,0.000,25.790,M-82_Week 1.pdf,NaN,6,WAR201030TH04,Week01
2,2021-08-03,160120,Norbord Inc - Huguley,PPW,25.32,0.000,25.320,M-82_Week 1.pdf,,6,WAR201030TH04,Week01
3,2021-08-03,160155,Norbord Inc - Huguley,PPW,25.31,0.000,25.310,M-82_Week 1.pdf,,6,WAR201030TH04,Week01
4,2021-08-03,160082,Norbord Inc - Huguley,PPW,27.17,0.000,27.170,M-82_Week 1.pdf,,6,WAR201030TH04,Week01
...,...,...,...,...,...,...,...,...,...,...,...,...
1569,2024-08-16,292262,West Fraser - Huguley,PPW,27.93,0.000,27.930,13093 week 20 pine beetle salvage (Flint Crew...,NaN,9,WAR240618BI03,Week10
1570,2024-08-17,292264,West Fraser - Huguley,PPW,29.21,0.000,29.210,13093 week 20 pine beetle salvage (Flint Crew...,NaN,9,WAR240618BI03,Week10
1571,2024-08-16,69692,Interfor - Thomaston,PST,31.24,0.125,31.115,13093 week 20 pine beetle salvage (Flint Crew...,NaN,9,WAR240618BI03,Week10
1572,2024-08-16,69753,Interfor - Thomaston,PST,27.99,0.000,27.990,13093 week 20 pine beetle salvage (Flint Crew...,NaN,9,WAR240618BI03,Week10


In [58]:
guidance_list = status_df['GuidanceProvider'].unique().tolist()
guidance_dict = {}
guidance_dict['guidance_provider'] = guidance_list
guidance_df = pd.DataFrame(guidance_dict)
guidance_df['guidance_uid'] = guidance_df.index
guidance_mapper = dict(zip(guidance_df['guidance_provider'], guidance_df['guidance_uid']))
status_df['guidance_provider'] = status_df['GuidanceProvider'].map(guidance_mapper)
status_df.drop(columns=['GuidanceProvider'], inplace=True)


In [59]:
status_df

,UniqueID,Guidance,Expiration,Status,Completed Date,guidance_provider
0,WAR241025TT01,,,PENDING,,0
1,WAR241025HD17,,,PENDING,,0
2,WAR241025HD16,,,PENDING,,0
3,WAR241025HD15,,,PENDING,,0
4,WAR241025HD14,,,PENDING,,0
...,...,...,...,...,...,...
589,WAR180518TH01,2018-05-18,,COMPLETED,,2
590,WAR180206TH04,2018-02-06,,DUPLICATE REQUEST,,2
591,WAR180206TH03,2018-02-06,,DUPLICATE REQUEST,,2
592,WAR180206TH02,2018-02-06,,COMPLETED,,2


In [60]:
guidance_df

,guidance_provider,guidance_uid
0,GA,0
1,ARB Guidance,1
2,,2


In [61]:
trial_mill_df = mill_df.copy()
trial_mill_df['ticket_uid'] = pd.to_numeric(trial_mill_df['TicketID'].map(ticket_mapper), errors='coerce')

In [62]:
ticketid_lookup_table[ticketid_lookup_table['TicketUID'] == 12]

,UniqueID,TicketUID
13,WAR230818TH01,12
14,WAR241025HD03,12


In [63]:
trial_mill_df[trial_mill_df['ticket_uid'].isna()]['TicketID'].unique().tolist()

['WAR230818TH01']

In [64]:
# Force it to work
trial_mill_df.loc[trial_mill_df['TicketID'] == 'WAR230818TH01', 'ticket_uid'] = 12

In [65]:
trial_mill_df.drop(columns=['TicketID'], inplace=True)
trial_mill_df['ticket_uid'] = trial_mill_df['ticket_uid'].astype(int)

In [66]:
trial_mill_df

,Date,Ticket,Mill,Product,Tons (U.S),Cull Tons,Accepted Tons,Source,Notes,Reporting Period,Week,ticket_uid
0,2021-08-02,160048,Norbord Inc - Huguley,PPW,26.31,0.000,26.310,M-82_Week 1.pdf,NaN,6,Week01,123
1,2021-08-03,160157,Norbord Inc - Huguley,PPW,25.79,0.000,25.790,M-82_Week 1.pdf,NaN,6,Week01,123
2,2021-08-03,160120,Norbord Inc - Huguley,PPW,25.32,0.000,25.320,M-82_Week 1.pdf,,6,Week01,123
3,2021-08-03,160155,Norbord Inc - Huguley,PPW,25.31,0.000,25.310,M-82_Week 1.pdf,,6,Week01,123
4,2021-08-03,160082,Norbord Inc - Huguley,PPW,27.17,0.000,27.170,M-82_Week 1.pdf,,6,Week01,123
...,...,...,...,...,...,...,...,...,...,...,...,...
1569,2024-08-16,292262,West Fraser - Huguley,PPW,27.93,0.000,27.930,13093 week 20 pine beetle salvage (Flint Crew...,NaN,9,Week10,47
1570,2024-08-17,292264,West Fraser - Huguley,PPW,29.21,0.000,29.210,13093 week 20 pine beetle salvage (Flint Crew...,NaN,9,Week10,47
1571,2024-08-16,69692,Interfor - Thomaston,PST,31.24,0.125,31.115,13093 week 20 pine beetle salvage (Flint Crew...,NaN,9,Week10,47
1572,2024-08-16,69753,Interfor - Thomaston,PST,27.99,0.000,27.990,13093 week 20 pine beetle salvage (Flint Crew...,NaN,9,Week10,47


## Try making a db and populating it with these tables

In [ ]:
import sqlite3

In [ ]:
conn = sqlite3.connect('./WAR_RP9_InProcess/trial-db/Harvest.db')
c = conn.cursor()

First I need to work on naming conventions and making sure everything has a primary key

In [ ]:
client_df.columns = [x.lower() for x in client_df.columns]
client_df.columns = [x.replace(' ', '_') for x in client_df.columns]
# client_df['id'] = client_df['uniqueid']
# client_df.drop(columns=['uniqueid'],inplace=True)
client_df = client_df[['id', 'tract', 'cmp', 'stand', 'acres', 'age', 'county', 'state', 'stand_type', 'origin', 'scheduled', 'preba', 'pretpa', 'pregta',
                       'plnba', 'plntpa', 'plngta', 'client_gt', 'future_gt', 'reporting_period']]

In [ ]:
client_df['acres'] = pd.to_numeric(client_df['acres'], errors='coerce').fillna(0)

In [ ]:
client_df.to_sql('client_intake', conn, if_exists='replace',index=False, dtype={'id': 'TEXT PRIMARY KEY'})

In [ ]:
ga_intake_df.columns = [x.lower().replace(' ', '_') for x in ga_intake_df.columns]
ga_intake_df.rename(columns={'uniqueid': 'id'},inplace=True)
ga_intake_df = ga_intake_df[['id', 'intakeid', 'folderid', 'ga_acres', 'ga_gt', 'total_gt']]

In [ ]:
ga_intake_df.to_sql('ga_intake', conn, if_exists='replace', index=False, dtype={'id': 'TEXT PRIMARY KEY'})

In [ ]:
ticketid_lookup_table.rename(columns={'TicketUID': 'id',
                                      'UniqueID': 'activity_id'}, inplace=True)
ticketid_lookup_table.to_sql('ticketid_lookup_activityid', conn, if_exists='replace', index=False, dtype={'activityid': 'TEXT PRIMARY KEY'})

In [ ]:
ticket_id_df.rename(columns={'TicketUID': 'id',
                             'TicketID' : 'ticketid'},inplace=True)
ticket_id_df.to_sql('ticketid', conn, if_exists='replace', index=False, dtype={'id': 'INTEGER PRIMARY KEY'})

In [ ]:
species_df.rename(columns={"SpeciesUID": "id",
                           'Species': 'species'}, inplace=True)
species_df.to_sql('species', conn, if_exists='replace', index=False, dtype={'id': 'INTEGER PRIMARY KEY'})

In [ ]:
origin_df.rename(columns={'OriginUID': 'id',
                          'Origin': 'origin'}, inplace=True)
origin_df.to_sql('origin', conn, if_exists='replace', index=False, dtype={'id': 'INTEGER PRIMARY KEY'})

In [ ]:
volume_expected_df = pd.read_csv('volume_expected_table_241030.csv')
volume_expected_df['id'] = volume_expected_df.index
volume_expected_df['trial_tickid'] = volume_expected_df['TicketID'].map(ticket_mapper)
volume_expected_df.drop(columns=['TicketID'], inplace=True)
volume_expected_df.rename(columns={'trial_tickid': 'ticketid',
                                   'Reporting Period': 'reporting_period',
                                   'Current RP Expected GT': 'expected_gt'},inplace=True)
# Have to force one ticketid because of a recent harvest
volume_expected_df['ticketid'].fillna(12, inplace=True)
volume_expected_df.to_sql('volume_expected', conn, if_exists='replace', index=False, dtype={'id': 'INTEGER PRIMARY KEY',
                                                                                            'ticketid': 'INTEGER'})

In [69]:
# Time for the mill ticket normalization
# Normalize the Weeks column.
# Normalize the source column
# Normalize the Mill column
# Normalize the Product column
trial_mill_df['Mill'].unique().tolist()

['Norbord Inc - Huguley',
 'WestRock Coated Board LLC - Mahrt',
 'International Paper 70980 - Rome Mill - New Fiber',
 'Rayonier Performance Fibers, LLC - Barnesville Fiber',
 'Interfor - Thomaston',
 'International Paper 70160 - Rome Wet Yard Fiber',
 'International Paper 30980 - Prattville - New Fiber',
 'Georgia-Pacific Corporation - Rome Lumber',
 'International Paper 70161 - Whitesburg Chip Mill - New Fiber',
 'Central Alabama Wood Products LLC',
 'International Paper 70981 - Flint River Mill - New Fiber',
 'Graphic Packaging International, Macon GA',
 'Jordan Forest Products, LLC',
 'International Paper 30312 - Dry Creek Woodyard - New Fiber',
 'West Fraser - Opelika',
 'Baseline Forest Services - Roanoke Woodyard',
 'Interfor - Preston',
 'International Paper 70160 - Rome Wet Yard - New Fiber',
 'Mountain Oak Mill',
 'WestRock Coated Board, LLC - Cottonton Saw Mill',
 'Dickert Forest Products',
 'Smith Mill',
 'Burt Lumber Co.',
 'Huber Engineered Woods LLC',
 'The Timbermen, In

In [70]:
trial_mill_df['Product'].unique().tolist()

['PPW',
 'PST',
 'CNS',
 'HPW',
 'HST',
 'HWP',
 'CTL',
 'Poles/Pilings',
 'Poles',
 'Posts',
 'PLY',
 'Chips',
 'Pulpwood',
 'Post',
 'PCNS',
 'SWPW',
 'SWLOG']